In [1]:
!pip install --upgrade astrapy pandas

from astrapy import DataAPIClient
import pandas as pd


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.5/300.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 17.0 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [2]:
from astrapy import DataAPIClient

# Initialize the client
client = DataAPIClient("AstraCS:ycCqpHAvXZnAUOrRQZQdkdMw:98132933d955132d8824c1f208242e5a90362f299ee97e34c9d620fb531bd948")
db = client.get_database_by_api_endpoint(
  "https://47bfa723-13db-41da-a8e1-3232e7c367f2-us-east-2.apps.astra.datastax.com"
)


silver = db.get_collection("sales_silver")

# Create Gold Collections
gold_region = db.create_collection("sales_gold_region")
gold_product = db.create_collection("sales_gold_product")
gold_year = db.create_collection("sales_gold_year")

print("Connected to Astra DB Gold Layer")

Connected to Astra DB Gold Layer


In [3]:
data = silver.find()
df = pd.DataFrame(data)
df.head()


,_id,order_id,region,country,item_type,sales_channel,order_priority,order_date,ship_date,units_sold,unit_price,unit_cost,total_revenue,total_cost,total_profit
0,19df6968-2861-4f8d-9f69-682861ff8d31,667593514,Middle East and North Africa,Morocco,Clothes,Online,M,2013-09-14T00:00:00.000Z,2013-10-19T00:00:00.000Z,4611,109.28,35.84,503890.08,165258.24,338631.84
1,a0e6a158-2289-40cb-a6a1-58228930cba5,752525556,Europe,Albania,Baby Food,Offline,H,2012-07-19T00:00:00.000Z,2012-08-13T00:00:00.000Z,7890,255.28,159.42,2014159.20,1257823.80,756335.40
2,252d400e-90fb-41e3-ad40-0e90fb21e33d,156530129,Sub-Saharan Africa,Tanzania,Fruits,Offline,H,2014-10-20T00:00:00.000Z,2014-11-29T00:00:00.000Z,9599,9.33,6.92,89558.67,66425.08,23133.59
3,769a5bf0-6aa7-4fe3-9a5b-f06aa72fe3c0,156295812,Australia and Oceania,East Timor,Cereal,Offline,M,2015-12-07T00:00:00.000Z,2015-12-30T00:00:00.000Z,259,205.70,117.11,53276.30,30331.49,22944.81
4,049fa0ee-e33e-4bc1-9fa0-eee33e8bc1fe,324669444,Europe,France,Cosmetics,Online,M,2015-12-07T00:00:00.000Z,2016-01-18T00:00:00.000Z,5758,437.20,263.33,2517397.60,1516254.14,1001143.46


In [4]:
agg1 = df.groupby('region')['total_revenue'].sum().reset_index()

for _, row in agg1.iterrows():
    gold_region.insert_one(row.to_dict())


In [5]:
agg2 = df.groupby('item_type')['total_profit'].sum().reset_index()

for _, row in agg2.iterrows():
    gold_product.insert_one(row.to_dict())


In [7]:
df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')


In [10]:
df_region = pd.DataFrame(gold_region.find())
df_region


,_id,region,total_revenue
0,046e2892-51cc-435b-ae28-9251cc635b04,Australia and Oceania,10711258.13
1,a1ba8a66-6bda-4676-ba8a-666bdaa6762c,Asia,28840812.19
2,5d1799b2-9aef-48f9-9799-b29aef58f978,Middle East and North Africa,24765127.25
3,b1a874c6-5676-41c4-a874-c6567641c478,Central America and the Caribbean,17570835.42
4,a7ce0ab0-6688-4833-8e0a-b06688183375,Europe,34964749.83
5,4dd7070e-6633-4f20-9707-0e6633af208a,Sub-Saharan Africa,24225437.42
6,ce7a92bd-15b0-44fd-ba92-bd15b0d4fdd5,North America,3611757.52


In [11]:
df_product = pd.DataFrame(gold_product.find())
df_product


,_id,item_type,total_profit
0,800ceaf1-bb61-4e16-8cea-f1bb61de16fd,Cosmetics,11424476.09
1,3d2e2a88-df6d-4ce4-ae2a-88df6d8ce4bc,Baby Food,1952859.92
2,2810b441-5767-49d6-90b4-415767b9d678,Meat,2884996.40
3,a8c61e6f-12da-4f1c-861e-6f12da5f1cc5,Office Supplies,5405267.50
4,8a581246-caee-4163-9812-46caeee163a3,Household,9552677.20
5,a2b41781-0ce1-4a35-b417-810ce11a35f2,Fruits,158867.20
6,b8a92dda-4788-4845-a92d-da4788c84535,Clothes,2948469.12
7,4a0abc3b-380f-4a79-8abc-3b380f2a792d,Personal Care,978467.70
8,0f758dbc-6b95-4cfb-b58d-bc6b95acfb0c,Vegetables,465141.84
9,9e2efb40-5647-476a-aefb-405647376a41,Cereal,4055295.84


In [22]:
print("Region Collection Count:", len(list(gold_region.find())))
print("Product Collection Count:", len(list(gold_product.find())))


Region Collection Count: 7
Product Collection Count: 12


In [20]:
gold_country = db.create_collection("sales_gold_country")

agg_country = df.groupby('country')['total_revenue'].sum().reset_index()

for _, row in agg_country.iterrows():
    gold_country.insert_one({
        'country': row['country'],
        'total_revenue': float(row['total_revenue'])
    })


In [21]:
df_country = pd.DataFrame(gold_country.find())
display(df_country)


,_id,country,total_revenue
0,776963a2-9fda-4344-a963-a29fdad34459,Ethiopia,289426.40
1,199ee03a-dc29-4fdd-9ee0-3adc29ffdd97,Malaysia,434357.30
2,c3a86d36-dd7c-4b3e-a86d-36dd7cbb3e50,Nicaragua,5944506.05
3,c83c7c3d-45f2-47f3-bc7c-3d45f2e7f38e,The Bahamas,790991.21
4,0803ac3b-32f2-4cc1-83ac-3b32f27cc140,Romania,1726589.35
...,...,...,...
75,36200044-0371-4062-a000-44037190620c,Malawi,2149107.66
76,760acf52-d688-4eac-8acf-52d688beac82,Mauritius,603225.60
77,6cea98ed-1085-46ad-aa98-ed108536add2,Belgium,231050.71
78,b2dce5e5-a1f4-4308-9ce5-e5a1f403087e,The Gambia,81161.67
